In [10]:
import cv2
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

In [11]:
import progressbar
class progressBar:
    def __init__(self):
            self.bar = progressbar.ProgressBar(maxval=1, \
                                  widgets=[progressbar.Bar('=','[',']'), ' ', progressbar.Percentage()])
            self.bar.start()
    
    def update(self,value):
        self.bar.update(value)

    def restart(self):
        self.bar.start()

In [25]:
class VLAD:

    def __init__(self,x_train,x_test):
        self.x_train = x_train
        self.x_test = x_test


    def cluster_formation(self,number_of_clusters):
        print("Performing KMeans Clustering","-"*(100-len("Performing KMeans Clustering")))
        keypoints = []
        counter=1
        bar = progressBar()
        for image in self.x_train :
            # image = self.resize2SquareKeepingAspectRation(image,150)
            bar.update(counter/len(list(self.x_train)))
            image = cv2.resize(image,(150,150))
            image =cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
            surf = cv2.xfeatures2d.SURF_create()
            kp, descriptors = surf.detectAndCompute(image,None)
            keypoints.append(descriptors)
            counter+=1
        keypoints = np.concatenate(keypoints, axis=0)
        self.kmeans = KMeans(n_clusters = number_of_clusters).fit(keypoints)
        print("KMeans Clustering finished","-"*(100-len("KMeans Clustering finished")))
        return self.kmeans
    
    def vlad(self,descriptors,number_of_clusters):
        dist_sum = []
        for i in range(number_of_clusters):
            dist_sum.append(np.zeros(64))
        predict_kmeans = self.kmeans.predict(descriptors)
        cluster_centers = self.kmeans.cluster_centers_
        for i in range(len(descriptors)):
            dist_sum[predict_kmeans[i]] += (np.asarray(descriptors[i]) - cluster_centers[predict_kmeans[i]])
        dist_sum = np.asarray(dist_sum)
        V = dist_sum.flatten()
        V = V/np.sqrt(np.dot(V,V))
        return V

    
    def vlad_features(self,number_of_clusters):
        self.cluster_formation(number_of_clusters)
        print("Performing VLAD extraction","-"*(100-len("Performing VLAD extraction")))
        x_feat_train = []
        x_feat_test = []
        counter=1
        bar = progressBar()
        for n in range(2):
            if(n==0): dataset = self.x_train
            else: dataset = self.x_test
            for image in dataset:
                # image = self.resize2SquareKeepingAspectRation(image,150)
                bar.update(counter/(len(list(self.x_train))+len(list(self.x_test))))
                image = cv2.resize(image,(150,150))
                image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
                surf = cv2.xfeatures2d.SURF_create()
                kp, descriptors = surf.detectAndCompute(image,None)
#                 V = VLADL(descriptors,self.kmeans)
                V = self.vlad(descriptors,number_of_clusters)
                if(n==0): x_feat_train.append(V)
                else: x_feat_test.append(V)
                counter+=1
        print("VLAD extraction finished","-"*(100-len("VLAD extraction finished")))
        return (x_feat_train,x_feat_test)

    def resize2SquareKeepingAspectRation(self,img, size, interpolation = cv2.INTER_AREA):
        h, w = img.shape[:2]
        c = None if len(img.shape) < 3 else img.shape[2]
        if h == w: return cv2.resize(img, (size, size), interpolation)
        if h > w: dif = h
        else: dif = w
        x_pos = int((dif - w)/2.)
        y_pos = int((dif - h)/2.)
        if c is None:
            mask = np.zeros((dif, dif), dtype=img.dtype)
            mask[y_pos:y_pos+h, x_pos:x_pos+w] = img[:h, :w]
        else:
            mask = np.zeros((dif, dif, c), dtype=img.dtype)
            mask[y_pos:y_pos+h, x_pos:x_pos+w, :] = img[:h, :w, :]
        return cv2.resize(mask, (size, size), interpolation)    

In [29]:
class CIFAR10:
    
        def __init__(self,path,number_of_clusters):
            x_train, y_train = self.load_cfar10_batch(path,1)
            x_train = np.asarray(x_train)
            y_train = np.asarray(y_train)
            self.xTrain, self.xTest, self.yTrain, self.yTest = train_test_split(x_train, y_train, test_size = 0.2, random_state = 0)
            print("Number of Training Images: {}".format(x_train.shape[0]))
            vlad = VLAD(self.xTrain, self.xTest)
            (self.x_feat_train,self.x_feat_test) = vlad.vlad_features(number_of_clusters)
    
        def load_cfar10_batch(self,cifar10_dataset_folder_path, batch_id):
            for i in range(1,6):
                with open(cifar10_dataset_folder_path + '/data_batch_' + str(i), mode='rb') as file:
                    # note the encoding type is 'latin1'
                    batch = pickle.load(file, encoding='latin1')
                    
                feature = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
                label = batch['labels']
                
                feature = np.array(feature)
                label = np.array(label)
                if(i==1):    
                    features = feature
                    labels = label
                else:
                    features = np.vstack((features,feature))
                    labels = np.concatenate((labels,label))
            return features, labels
        
        def random_forest(self):
            print("Training Model","-"*(100-len("Training Model")))

            # self.rf =KNeighborsClassifier(n_neighbors=5, metric='euclidean')
            self.rf = RandomForestClassifier(n_estimators = 1000)
            self.rf.fit(self.x_feat_train, self.yTrain)

            y_pred = self.rf.predict(self.x_feat_test)
            accuracy = accuracy_score(y_pred, self.yTest)
            print("Acuuracy of Model: {}".format(accuracy))
            return confusion_matrix(self.yTest, y_pred)
        
        def SVM(self):
            self.clf = svm.LinearSVC(multi_class='ovr')
            self.clf.fit(self.x_feat_train, self.yTrain)

            y_pred = self.clf.predict(self.x_feat_test)
            accuracy = accuracy_score(y_pred, self.yTest)
            print("Acuuracy of Model: {}".format(accuracy))
            return confusion_matrix(self.yTest, y_pred)

In [30]:
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
x = CIFAR10("../assignment2/data/cifar-10-batches-py",60)

Number of Training Images: 50000
Performing KMeans Clustering ------------------------------------------------------------------------


KMeans Clustering finished --------------------------------------------------------------------------
Performing VLAD extraction --------------------------------------------------------------------------


VLAD extraction finished ----------------------------------------------------------------------------


In [31]:
from sklearn import svm
x.SVM()

Acuuracy of Model: 0.4281


array([[549,  49,  60,  25,  30,  31,  22,  60, 125,  53],
       [ 40, 550,  24,  31,  18,  38,  46,  38,  45, 156],
       [145,  50, 230,  74, 109,  96, 118,  89,  79,  26],
       [ 49,  68,  85, 198,  91, 166, 149,  80,  65,  79],
       [ 46,  32,  77,  74, 323,  53, 161, 114,  43,  44],
       [ 39,  45,  54, 107,  71, 361,  84,  96,  54, 105],
       [ 16,  42,  37,  62,  85,  56, 565,  31,  26,  33],
       [ 53,  52,  54,  56,  93,  75,  46, 432,  47,  91],
       [128,  81,  46,  26,  42,  43,  33,  35, 494,  61],
       [ 35, 138,  19,  44,  40,  36,  51,  50,  48, 579]])